In [1]:
import yaml
from pathlib import Path
from aiida import orm, load_profile

load_profile()

Profile<uuid='960a408e945248238ac64a4982cfc6d7' name='dev'>

In [12]:
def find_source_node(group_label, source_id, source_db=None):
    """Find the node corresponding to a certain source in a group."""

    filters = {"and": [{"extras.source.id": source_id}]}
    if source_db:
        filters["and"].append({"extras.source.db": source_db})

    try:
        return (
            orm.QueryBuilder()
            .append(orm.Group, filters={"label": group_label}, tag="group")
            .append(orm.Node, with_group="group", filters=filters)
            .first()[0]
        )
    except TypeError:
        return None

In [2]:
from cli import RelaxSubmissionController

with Path('settings.yaml').open("r") as handle:
    settings_dict = yaml.safe_load(handle)

submission_controller = RelaxSubmissionController(
    unique_extra_keys=["source.database", "source.id"],
    max_concurrent=settings_dict["max_concurrent"],
    parent_group_label=settings_dict["parent_group"],
    group_label=settings_dict["relax_group"],
    pw_code=settings_dict["pw_code"],
    overrides=settings_dict["overrides"],
    filters={'extras.number_of_sites': {'<=': settings_dict["max_structure_size"]}},
    order_by={orm.Node: {'extras.number_of_sites': {'order': 'asc', 'cast': 'i'}}}
)

In [14]:
naughty_structure = find_source_node('structure/source', 'S1816748')

In [22]:
pmg_structure = naughty_structure.get_pymatgen()
pmg_structure

Structure Summary
Lattice
    abc : 4.06827437469227 4.06827437469227 4.068274374692269
 angles : 59.45961601886717 59.45961601886717 59.459616018867194
 volume : 47.026638095898306
      A : 2.0175 1.1648041680900698 3.335233333333333
      B : -2.0175 1.1648041680900698 3.335233333333333
      C : -6.476300976980106e-19 -2.3296083361801396 3.335233333333333
    pbc : True True True
PeriodicSite: U (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Se (2.017, 1.165, 8.338) [1.5, 0.5, 0.5]

In [23]:
pmg_structure.translate_sites(range(len(pmg_structure)), [0, 0, 0], to_unit_cell=True)

Structure Summary
Lattice
    abc : 4.06827437469227 4.06827437469227 4.068274374692269
 angles : 59.45961601886717 59.45961601886717 59.459616018867194
 volume : 47.026638095898306
      A : 2.0175 1.1648041680900698 3.335233333333333
      B : -2.0175 1.1648041680900698 3.335233333333333
      C : -6.476300976980106e-19 -2.3296083361801396 3.335233333333333
    pbc : True True True
PeriodicSite: U (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Se (-4.444e-16, -6.661e-16, 5.003) [0.5, 0.5, 0.5]

In [9]:
len(submission_controller.get_all_extras_to_submit())

316

In [9]:
submission_controller.parent_group.nodes[0].extras

{'source': {'id': 'S1100556', 'version': '1.0.0', 'database': 'mpds'},
 'duplicates': [],
 'formula_hill': 'F8Sn4',
 'bravais_lattice': 'oP',
 'chemical_system': '-F-Sn-',
 'spacegroup_number': 19,
 'partial_occupancies': False,
 'formula_hill_compact': 'F2Sn',
 'bravais_lattice_extended': 'oP1',
 'spacegroup_international': 'P2_12_12_1'}

In [12]:
submission_controller.parent_group.nodes[0].attributes

{'cell': [[4.836, 0.0, 0.0], [0.0, 5.211, 0.0], [0.0, 0.0, 8.418]],
 'pbc1': True,
 'pbc2': True,
 'pbc3': True,
 'kinds': [{'mass': 118.71, 'name': 'Sn', 'symbols': ['Sn'], 'weights': [1.0]},
  {'mass': 18.9984032, 'name': 'F', 'symbols': ['F'], 'weights': [1.0]}],
 'sites': [{'position': [1.131624, 2.6487513, 5.312599799999999],
   'kind_name': 'Sn'},
  {'position': [3.549624, 5.167748700000001, 3.1054001999999996],
   'kind_name': 'Sn'},
  {'position': [3.7043760000000003, 0.04325129999999987, 7.3144002],
   'kind_name': 'Sn'},
  {'position': [1.2863760000000002, 2.5622487000000005, 1.1035997999999998],
   'kind_name': 'Sn'},
  {'position': [2.280174, 4.7810925000000015, 4.3630493999999995],
   'kind_name': 'F'},
  {'position': [4.698174000000001, 3.035407499999999, 4.0549506],
   'kind_name': 'F'},
  {'position': [2.555826, 2.1755925000000014, 8.2639506], 'kind_name': 'F'},
  {'position': [0.13782599999999987, 0.42990749999999894, 0.15404939999999984],
   'kind_name': 'F'},
  {'pos

In [30]:
import aiida 
from aiida.orm import load_node
from aiida_quantumespresso.workflows.pw.bands import PwBandsWorkChain
from aiida_quantumespresso.common.types import ElectronicType, SpinType

aiida.load_profile()

Profile<uuid='960a408e945248238ac64a4982cfc6d7' name='dev'>

In [32]:
builder = PwBandsWorkChain.get_builder_from_protocol(
            code='sirius-pw@lumi',
            structure=load_node(21261),
            electronic_type=ElectronicType.METAL,
            spin_type=SpinType.COLLINEAR,
        )

In [33]:
builder

Process class: PwBandsWorkChain
Inputs:
bands:
  max_iterations: 5
  metadata: {}
  pw:
    code: sirius-pw@lumi
    metadata:
      options:
        max_wallclock_seconds: 43200
        resources:
          num_machines: 1
          num_mpiprocs_per_machine: 8
        stash: {}
        withmpi: true
    monitors: {}
    parameters:
      CONTROL:
        calculation: bands
        etot_conv_thr: 4.0e-05
        forc_conv_thr: 0.0001
        restart_mode: from_scratch
        tprnfor: true
        tstress: true
      ELECTRONS:
        conv_thr: 8.0e-10
        diago_full_acc: true
        diagonalization: paro
        electron_maxstep: 80
        mixing_beta: 0.4
        startingpot: file
      SYSTEM:
        degauss: 0.02
        ecutrho: 480.0
        ecutwfc: 60.0
        nosym: false
        nspin: 2
        occupations: smearing
        smearing: cold
        starting_magnetization:
          H: 0.1
          Sc: 0.45454545454545453
    pseudos:
      H: ''
      Sc: ''
bands_kp

In [34]:
if 'relax' in builder:
    del builder['relax']

builder

Process class: PwBandsWorkChain
Inputs:
bands:
  max_iterations: 5
  metadata: {}
  pw:
    code: sirius-pw@lumi
    metadata:
      options:
        max_wallclock_seconds: 43200
        resources:
          num_machines: 1
          num_mpiprocs_per_machine: 8
        stash: {}
        withmpi: true
    monitors: {}
    parameters:
      CONTROL:
        calculation: bands
        etot_conv_thr: 4.0e-05
        forc_conv_thr: 0.0001
        restart_mode: from_scratch
        tprnfor: true
        tstress: true
      ELECTRONS:
        conv_thr: 8.0e-10
        diago_full_acc: true
        diagonalization: paro
        electron_maxstep: 80
        mixing_beta: 0.4
        startingpot: file
      SYSTEM:
        degauss: 0.02
        ecutrho: 480.0
        ecutwfc: 60.0
        nosym: false
        nspin: 2
        occupations: smearing
        smearing: cold
        starting_magnetization:
          H: 0.1
          Sc: 0.45454545454545453
    pseudos:
      H: ''
      Sc: ''
bands_kp